# Simple System Architecture: Game & OS Agent

**Goal:** Build a fast, responsive game that works on desktop and can be easily moved to the web, keeping things separate and safe.

---

## The Game (One Program, Many Workers)

The game runs as a single application, but it divides work among several internal workers (**threads**) to avoid slowdowns.

| Worker (Thread) | Job | Rule |
| :--- | :--- | :--- |
| **GameLoop** | Draws the screen, takes player input, and plays audio. | **CRITICAL:** Must run smoothly. **Never** wait for slow tasks here. |
| **Diary** | Reads the book/file content (slow I/O). | Keeps file loading separate from gameplay. |
| **Chatbot** | Calculates AI replies (slow computation). | Uses **message boxes (queues)** to deliver results without freezing the game. |
| **Entity Brain** | Decides *what* needs to be done (e.g., "save a note"). | It asks for an action, but doesn't do the system work itself. |

---

## The OS Agent (A Separate, Safe Assistant)

A second, completely separate program handles all risky or sensitive access to the computer.

* **Communication:** Talks to the game using a secure text-based channel (**IPC/JSON**).
* **Safety Check:** Only allows a **pre-approved list** of commands (e.g., "show a pop-up," "write a log file"). **Blocks everything else** (like running system commands).

### Security and Reliability

1.  **Isolation:** If the main game has a crash or security problem, it **cannot harm the system** because the Agent is isolated in a separate process.
2.  **Safety Modes:**
    * **Safe Mode (Default):** Pretends to do the action (uses **mocks**); no system access.
    * **Enhanced Mode (Opt-in):** Performs real actions, but with **minimum permissions** and logging.
3.  **Crash Proofing:** If the Agent crashes, the game switches to **Safe Mode** instantly and keeps running (a **circuit breaker**).

### Web Readiness

* To move to the web, we simply swap the **OS Agent program** with a **Web Worker** that calls browser APIs (like notifications).
* The game itself doesn't change—it just sends the same requests to a different backend.

---

**In short:** Use internal workers (threads) for speed, and a separate helper program (OS Agent) for security and stability.